In [1]:
import pandas as pd
import pyarrow as pa
!pip install --upgrade python-dotenv
!pip install "python-dotenv>=1.0.1,<2.0.0"
!pip show python-dotenv pymilvus
!pip uninstall backports.tarfile -y
!pip install backports.tarfile
!pip install langchain langchain-core langchain-azure-ai
! pip install datasets langchain-huggingface langchain-milvus
# LangChain core and modules
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_milvus import Milvus
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA
import pandas as pd
import pyarrow.parquet as pq
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_azure_ai.embeddings import AzureAIEmbeddingsModel
from langchain_milvus import Milvus
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
!pip install rank_bm25



Name: python-dotenv
Version: 1.0.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: https://github.com/theskumar/python-dotenv
Author: Saurabh Kumar
Author-email: me+github@saurabh-kumar.com
License: BSD-3-Clause
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: 
Required-by: anaconda-cloud-auth, pydantic-settings, pymilvus
---
Name: pymilvus
Version: 2.5.4
Summary: Python Sdk for Milvus
Home-page: 
Author: 
Author-email: Milvus Team <milvus-team@zilliz.com>
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: grpcio, milvus-lite, pandas, protobuf, python-dotenv, setuptools, ujson
Required-by: langchain-milvus
Found existing installation: backports.tarfile 1.2.0
Uninstalling backports.tarfile-1.2.0:
  Successfully uninstalled backports.tarfile-1.2.0
  Using cached backports.tarfile-1.2.0-py3-none-any.whl.metadata (2.0 kB)
Using cached backports.tarfile-1.2.0-py3-none-any.whl (30 kB)


In [2]:
API_ENDPOINT = "http://81.171.3.27:65432/models"
API_KEY = "g7Ma8ocYuhTBasjL"

In [11]:
import pandas as pd
import pyarrow.parquet as pq
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_azure_ai.embeddings import AzureAIEmbeddingsModel
from langchain_milvus import Milvus
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.retrievers import BM25Retriever


chat_model = AzureAIChatCompletionsModel(endpoint=API_ENDPOINT, credential=API_KEY, model_name="DeepSeek-R1")
embeddings = AzureAIEmbeddingsModel(endpoint=API_ENDPOINT, credential=API_KEY, model_name="text-embedding-3-small")

# ✅ Load Dataset & Vectorstore
df = pd.read_parquet("cahiers-du-foot.parquet")
vector_store = Milvus(
    embeddings,
    index_params={"index_type": "FLAT", "metric_type": "COSINE"},
    connection_args={"uri": "milvus-cahiers-du-foot.db"}
)

def is_english(text):
    """Checks if the input query is in English using DeepSeek-R1."""
    messages = [
        SystemMessage(content="You are a language detection assistant. "
                              "Return only 'YES' if the text is in English, otherwise return 'NO'."),
        HumanMessage(content=text)
    ]
    
    response = chat_model.invoke(messages).content.strip().upper()

    # Ensure response is strictly "YES" or "NO"
    if response not in ["YES", "NO"]:
        return True  # Assume English if uncertain

    return response == "YES"


# ✅ Contextual Query Generation
def contextual_query_generation(query, conversation_history):
    """
    Pre-retrieval: Generates a more precise query using conversation context.
    """
    messages = [
        SystemMessage(content="You are an intelligent assistant. Given the user's query and conversation history, "
                              "refine the query to be more specific for document retrieval."),
        HumanMessage(content=f"User Query: \"{query}\"\nConversation History: \"{conversation_history}\"")
    ]
    refined_query = chat_model.invoke(messages).content.strip()
    print(f"[DEBUG] Refined Query: {refined_query}")
    return refined_query

# ✅ Adaptive Retrieval
def adaptive_retrieval(query):
    """
    Adaptive retrieval: Selects different numbers of candidate documents based on query complexity.
    Uses a vector database for initial retrieval.
    """
    k = 8 if len(query.split()) >= 5 else 5  # More documents for longer queries
    print(f"[DEBUG] Adaptive retrieval using k = {k}")
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": k})
    docs = retriever.invoke(query)
    print(f"[DEBUG] Retrieved {len(docs)} candidate documents.")
    return docs
    
# ✅ Function to Translate English Query to French
def translate_english_to_french(english_text):
    """Translates an English query into French using Mistral-Nemo."""
    messages = [
        SystemMessage(content="Translate the following English text into fluent French."),
        HumanMessage(content=english_text)
    ]
    mistral_model = AzureAIChatCompletionsModel(endpoint=API_ENDPOINT, credential=API_KEY, model_name="Mistral-Nemo")
    return mistral_model.invoke(messages).content

# ✅ Function to Retrieve Documents in French
def retrieve_documents(query, top_k=5):
    """Retrieves the top_k most relevant documents in French."""
    query_embedding = embedding_model.embed_query(query)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
    return retriever.invoke(query)

# ✅ Function to Translate French Documents to English
def translate_french_to_english(french_text):
    """Translates a French document into English using Mistral-Nemo."""
    messages = [
        SystemMessage(content="Translate the following French text into fluent English."),
        HumanMessage(content=french_text)
    ]
    mistral_model = AzureAIChatCompletionsModel(endpoint=API_ENDPOINT, credential=API_KEY, model_name="Mistral-Nemo")
    return mistral_model.invoke(messages).content

# ✅ Function to Ensure Answer is Based on Relevant Sources
def is_relevant_response(query, answer):
    """Verifies if the answer is relevant to the query using GPT-4o."""
    messages = [
        SystemMessage(content="Check if the provided answer is relevant to the user's query. "
                              "Return 'YES' if relevant, otherwise return 'I don't know'."),
        HumanMessage(content=f"Query: {query}\n\nAnswer: {answer}")
    ]
    response = chat_model.invoke(messages).content.strip()
    return response

# ✅ Function to Format Retrieved Documents
def format_documents_for_chat(docs):
    """Formats retrieved documents for LLM input."""
    formatted_text = "Here are the relevant documents:\n\n"
    for doc in docs:
        formatted_text += f"Title: {doc['title']}\n"
        formatted_text += f"URL: {doc['url']}\n"
        formatted_text += f"Content: {doc['content'][:500]}...\n\n"  # Limit length to avoid exceeding context limit
    return formatted_text


# ✅ Updated Response Generation with Contextual Query & Adaptive Retrieval
def generate_hybrid_response(query, conversation_history=""):

    # 🚫 Reject non-English queries
    if not is_english(query):
        return "Please provide your input in English."
        
    """Processes an English query, retrieves translated documents, and generates a response with sources."""

    # 🔍 Generate refined query using conversation history
    refined_query = contextual_query_generation(query, conversation_history)

    # 📖 Retrieve relevant documents (already in French)
    retrieved_docs = retrieve_documents(refined_query)

    # ❌ If no relevant documents, return "I don't know."
    if not retrieved_docs:
        return "I don't know."

    # 🔄 Translate retrieved French documents to English using Mistral
    translated_docs = [
        {
            "title": doc.metadata["title"],
            "url": doc.metadata["url"],
            "content": translate_french_to_english(doc.page_content)
        }
        for doc in retrieved_docs
    ]

    # 📝 Format documents for LLM input
    formatted_docs = format_documents_for_chat(translated_docs)

    # 🧠 Use DeepSeek-R1 for reasoning
    messages = [
        SystemMessage(content="You must answer only in English. "
                              "Only take English input, if any other language instantly reply please give English"
                              "Use only the retrieved documents. "
                             "remember chat history, and answer stuff relevant to that too"
                              "Translate them into English if needed. "
                              "If no relevant documents exist, reply with 'I don't know.' "
                              "In this case give no sources"
                              "Include sources in the response, unless answer is I don't know"
                              "Always give the links unless I dont know!!"),
        HumanMessage(content=f"User Query: {query}\n\nHere are the retrieved documents:\n\n{formatted_docs}")
    ]
    
    response = chat_model.invoke(messages).content

   
    return response 


# ✅ Inline User Interface (Simple Console Chatbot)
def chatbot_interface():
    """Runs an interactive chatbot session."""
    print("💬 Chatbot Ready! Type 'exit' to stop.")
    while True:
        query = input("\n🔹 You: ")
        if query.lower() == "exit":
            print("👋 Goodbye!")
            break
        response = generate_hybrid_response(query)
        print("\n🤖 Chatbot:", response)

# 🔥 Start Chatbot
chatbot_interface()


💬 Chatbot Ready! Type 'exit' to stop.



🔹 You:  que est ce


[DEBUG] Refined Query: <think>
Okay, let me try to figure out what the user is asking here. The query is "que est ce" which is French. Translating that, it should be "Qu'est-ce que..." which means "What is..." in English. But the user only wrote "que est ce" which is a bit broken. Maybe they meant to ask "Qu'est-ce que c'est?" which translates to "What is this?" or "What is that?".

Looking at the conversation history, it's empty. So there's no prior context to go off. The user might be starting a new conversation. Since the query is in French, I should consider that they're a French speaker or prefer French responses. But the query is incomplete, so they probably need help forming a proper question. 

They might be trying to ask "What is..." followed by something specific. Maybe they're unsure how to phrase their question correctly in French. For example, "Qu'est-ce que la philosophie?" (What is philosophy?) or "Qu'est-ce que le temps?" (What is time?). But without more context, it's 


🔹 You:  exit


👋 Goodbye!


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain_azure_ai.embeddings import AzureAIEmbeddingsModel
from langchain_milvus import Milvus
from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage



# ✅ Initialize GPT-4o Model
chat_model = AzureAIChatCompletionsModel(endpoint=API_ENDPOINT, credential=API_KEY, model_name="gpt-4o")

# ✅ Initialize Embedding Model
embedding_model = AzureAIEmbeddingsModel(endpoint=API_ENDPOINT, credential=API_KEY, model_name="text-embedding-3-small")

# ✅ Load Dataset & Vectorstore
df = pd.read_parquet("cahiers-du-foot.parquet")
vector_store = Milvus(
    embeddings,
    index_params={"index_type": "FLAT", "metric_type": "COSINE"},
    connection_args={"uri": "milvus-cahiers-du-foot.db"}
)

# ✅ Function to Retrieve Relevant Documents (in French)
def retrieve_documents(query, top_k=5):
    """Retrieves the top_k most relevant documents in French."""
    query_embedding = embedding_model.embed_query(query)
    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
    return retriever.invoke(query)

# ✅ Function to Generate a Response Using Only GPT-4o
def generate_response_gpt4(query):
    """Uses GPT-4o for translation, retrieval, and response generation in one step."""

    # 📖 Retrieve relevant documents (French)
    retrieved_docs = retrieve_documents(query)

    # ❌ If no relevant documents, return "I don't know."
    if not retrieved_docs:
        return "I don't know."

    # 📝 Format documents for input
    formatted_docs = "\n\n".join(
        [f"Title: {doc.metadata['title']}\nURL: {doc.metadata['url']}\nContent: {doc.page_content[:500]}..." for doc in retrieved_docs]
    )

    # 🤖 Use GPT-4o for full pipeline (Translation + Reasoning)
    messages = [
        SystemMessage(content="Only take a response in english, if not return please give input in English"
                              "You must answer only in English. "
                              
                              "Use only the retrieved documents. "
                             "remember chat history, and answer stuff relevant to that too"
                              "Translate them into English if needed. "
                              "If no relevant documents exist, reply with 'I don't know.' "
                              "Always include sources in the response, unless answer is I don't know"
                              "Always give the links unless I dont know!!"),
        HumanMessage(content=f"User Query: {query}\n\nHere are the retrieved documents:\n\n{formatted_docs}")
    ]
    
    response = chat_model.invoke(messages).content

    
    return response

# ✅ Simple Chat Interface
def chatbot_interface():
    """Runs an interactive chatbot session using GPT-4o."""
    print("💬 GPT-4o Chatbot Ready! Type 'exit' to stop.")
    while True:
        query = input("\n🔹 You: ")
        if query.lower() == "exit":
            print("👋 Goodbye!")
            break
        response = generate_response_gpt4(query)
        print("\n🤖 Chatbot:", response)

# 🔥 Start Chatbot
chatbot_interface()


💬 GPT-4o Chatbot Ready! Type 'exit' to stop.
